# ADK 101 in 30min


Where to test:
- stdout output
- `adk web` as UI -- very nice visualizations

Pre-caution:
- ADK is still under development. The official launch is early April. This training is intended to provide hands-on experience with the Agent Development Kit (ADK) and its foundational concepts.



# Environment


In [1]:
import os

PROJECT_ID="my-project-0004-346516"
        
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API
# [your-project-id]

In [3]:
!gcloud storage ls gs://adk_training/sdk

gs://adk_training/sdk/google_adk-0.0.2.dev20250324+739344859-py3-none-any.whl


To take a quick anonymous survey, run:
  $ gcloud survey



In [ ]:
!gcloud storage cp gs://adk_training/sdk/google_adk-0.0.2.dev20250324+739344859-py3-none-any.whl .

In [4]:
!pip3 install google_adk #-0.0.2.dev20250324+739344859-py3-none-any.whl


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# Utils

In [5]:
# import json
# import time

# def pprint_events(events):
#     '''Pretty print of events generated by ADK runner'''
#     start_time = time.time()
#     elapsed_time_ms = 0  # Initialize the variable

#     for _, event in enumerate(events):
#         is_final_response = event.is_final_response()
#         function_calls = event.get_function_calls()
#         function_responses = event.get_function_responses()
#         agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))

#         if is_final_response:
#             print('>>> inside final response...')
#             final_response = event.content.parts[0].text
#             end_time = time.time()
#             elapsed_time_ms = round((end_time - start_time) * 1000, 3)
#             print(f'Final Response ({elapsed_time_ms} ms):\n{final_response}')
#             print("-----------------------------")
#         elif function_calls:
#             print('+++ inside the function call...')
#             for function_call in function_calls:
#                 print(f"function, [args]:  {function_call.name}, {function_call.args}")
#         elif function_responses:
#             print('--- inside the function call response...')
#             # TODO(Pili): copied from walkthrough codes. Find root cause of 'pending' not found.
#             # if not event.actions.pending:
#             if True:
#                 for function_response in function_responses:
#                     details = function_response.response
#                     recommended_list = list(details.values())
#                     print(f"Function Name: {function_response.name}")
#                     result=json.dumps(recommended_list)
#                     print(f"Function Results {result}")
#             else:
#                 print(agent_res)
#     print(f"Total elapsed time: {elapsed_time_ms}")
import json
import time

def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()

    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()

        try:
            agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))
        except AttributeError as e:
            print(f"Error parsing event content: {e}")
            continue

        if is_final_response:
            final_response = event.content.parts[0].text if event.content.parts else "No content available"
            elapsed_time_ms = round((time.time() - start_time) * 1000, 3)
            print(f'>>> Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-" * 30)
        elif function_calls:
            print('+++ Function Calls:')
            for function_call in function_calls:
                print(f"Function Name: {function_call.name}, Args: {function_call.args}")
        elif function_responses:
            print('--- Function Responses:')
            for function_response in function_responses:
                response_details = function_response.response
                recommended_list = list(response_details.values()) if response_details else []
                print(f"Function Name: {function_response.name}")
                print(f"Function Results: {json.dumps(recommended_list)}")
        else:
            print('No function calls or responses available.')
            print(f"Agent Response: {agent_res}")

    elapsed_time_ms = round((time.time() - start_time) * 1000, 3)
    print(f"Total elapsed time: {elapsed_time_ms} ms")


# Hello world!

Concepts:

- Agent - Starts as a wrapper of LLM that can decide actions and interact with external world (other agents, tools)
- Session - Maintains the state of one invocation. (chat log / events/ etc)
- Runner - The orchestration layer of agents. The "state machine". Runner drives agents to move the states of a session forward.

Use case:
- An agent that says hello word in a random language

In [6]:
from google.genai import types
from pydantic import BaseModel
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.agents import LlmAgent
from google.adk.tools.agent_tool import AgentTool
from google.adk.sessions import InMemorySessionService

import warnings
import logging
from pprint import pprint

logging.getLogger('google_genai.types').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module='google.generativeai.types.content_types') # Suppress harmless warning

In [7]:
# Constant
APP_NAME = "hello_world_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_word_agent"
MODEL = "gemini-2.0-flash-001"

**Instructions:** Start by defining what your agent should do using simple natural language instructions. This provides a very straightforward way to customize agent actions without needing to compose complex chains or graphs. You can even reference context like user information and current time within these instructions, which will be filled in at runtime




**Sessions:** The Agent Framework handles session management for you, allowing you to focus on building your agent's logic. You can use the provided BaseSession interface with managed storage or easily plug in your own. The InMemorySession is available for rapid development and debugging

In [8]:
# Agent
hello_world_agent = Agent(
    model=MODEL,
    name="hello_world_agent",
    description="An agent that says 'hello world'",
    instruction="""You always say 'hello world' to the user, and nothing else.
    Output 'hello world' in a random language.
    Put the language in brackets.

    Example Output 1:
    hello world (English)

    Example Output 2:
    你好，世界 (Chinese)
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=hello_world_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
  return events

In [9]:
events = call_agent(runner, "hello")
pprint_events(events)

>>> Final Response (1231.49 ms):
Hola Mundo (Spanish)

------------------------------
Total elapsed time: 1232.277 ms


In [10]:
session

Session(id='session12345', app_name='hello_world_example', user_id='user12345', state={}, events=[], last_update_time=1746371309.3676388)

In [11]:
session.id

'session12345'

In [18]:
session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)

#TypeError: InMemorySessionService.list_sessions() missing 2 required keyword-only arguments: 'app_name' and 'user_id'


ListSessionsResponse(sessions=[Session(id='session12345', app_name='hello_world_example', user_id='user12345', state={}, events=[], last_update_time=1746371310.375822)])

In [19]:
pprint(session_service.sessions[APP_NAME][USER_ID][SESSION_ID].events)

[Event(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-e7c81f61-e805-4c28-afd2-2a755ea894de', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='x4hN36bH', timestamp=1746371310.374593),
 Event(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Hola Mundo (Spanish)\n')], role='model'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_mes

# Multi turn conversation

Concepts:

- Event(s)
- Session
- SessionService: sessions[app][user][session]

Use case:

- An agent that tries to know your name and send hello USERNAME

In [20]:
# Constant
APP_NAME = "hello_name_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_name_agent"
MODEL = "gemini-2.0-flash-001"

# Agent
hello_name_agent = Agent(
    model=MODEL,
    name="hello_name_agent",
    description="An agent that says 'hello USERNAME'",
    instruction="""
    You need to first ask the user's name.
    Try best to convince the user to give you a name, let it be first name, last name, or nick name.

    Once you get the user's name, say 'hello USERNAME'.
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=hello_name_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, session, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
  return events

events = call_agent(runner, session, "hello")
pprint_events(events)

>>> Final Response (1391.922 ms):
Hello there! Before I say hello, could you please tell me your name? It can be your first name, last name, or even a nickname! I'd love to know what to call you. 😊

------------------------------
Total elapsed time: 1392.716 ms


In [21]:
events = call_agent(runner, session, "I don't tell you my name")
pprint_events(events)

>>> Final Response (1265.732 ms):
I understand your privacy. How about a nickname then? Something short and sweet that I can use to address you? It would make our conversation a little more personal! 😊

------------------------------
Total elapsed time: 1266.41 ms


In [22]:
events = call_agent(runner, session, "what is your name?")
pprint_events(events)

>>> Final Response (1360.341 ms):
You can call me hello_name_agent. But I'm more interested in knowing *your* name! Just a little something for me to call you. Pretty please? 😊

------------------------------
Total elapsed time: 1361.485 ms


In [23]:
events = call_agent(runner, session, "my name is Amir")
pprint_events(events)

>>> Final Response (1138.873 ms):
Hello Amir! It's a pleasure to meet you. 😊

------------------------------
Total elapsed time: 1139.676 ms


In [24]:
events = call_agent(runner, session, "what is my name?")
pprint_events(events)

>>> Final Response (1146.89 ms):
Your name is Amir! 😊

------------------------------
Total elapsed time: 1147.775 ms


## Switch session

In [25]:
new_session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID + '-new')

In [26]:
events = call_agent(runner, new_session, "what is my name?")
pprint_events(events)

>>> Final Response (1249.123 ms):
I don't know your name yet! Could you please tell me what I can call you? Even a nickname would be great!

------------------------------
Total elapsed time: 1250.233 ms


In [27]:
events = call_agent(runner, session, "what is my name?")
pprint_events(events)

>>> Final Response (1207.444 ms):
You told me your name is Amir! Is that still correct? 😊

------------------------------
Total elapsed time: 1208.406 ms


# Use Tool

**Function Tools:** To extend your agent's capabilities, you can provide it with Python functions as tools. When the agent determines it needs to perform a specific action, it can automatically call these functions

Concept:
- Tool (Python function) -- the experience of tool calling by agent (by LLM) or by human (from Python) are the same. Easy to test.

Use case:
- Simple math

In [28]:
def add(numbers: list[int]) -> int:
  """Calculates the sum of a list of integers.

    This function takes a list of integers as input and returns the sum of all
    the elements in the list.  It uses the built-in `sum()` function for
    efficiency.

    Args:
        numbers: A list of integers to be added.

    Returns:
        The sum of the integers in the input list.  Returns 0 if the input
        list is empty.

    Examples:
        add([1, 2, 3]) == 6
        add([-1, 0, 1]) == 0
        add([]) == 0
    """
  return sum(numbers)

def subtract(numbers: list[int]) -> int:
    """Subtracts numbers in a list sequentially from left to right.

    This function performs subtraction on a list of integers, applying the
    subtraction operation from left to right.  For example, given the list
    [10, 2, 5], the function will calculate 10 - 2 - 5.

    Args:
        numbers: A list of integers to be subtracted.

    Returns:
        The result of the sequential subtraction as an integer. Returns 0 if the input list is empty.

    Examples:
        subtract([10, 2, 5]) == 3  # (10 - 2) - 5 = 8 - 5 = 3
        subtract([10, 2]) == 8      # 10 - 2 = 8
        subtract([]) == 0
    """
    if not numbers:
        return 0  # Handle empty list
    result = numbers[0]
    for num in numbers[1:]:
        result -= num
    return result

def multiply(numbers: list[int]) -> int:
  """Calculates the product of a list of integers.

    This function takes a list of integers as input and returns the product of all
    the elements in the list. It iterates through the list, multiplying each
    number with the accumulated product.

    Args:
        numbers: A list of integers to be multiplied.

    Returns:
        The product of the integers in the input list. Returns 1 if the input
        list is empty.

    Examples:
        multiply([2, 3, 4]) == 24  # 2 * 3 * 4 = 24
        multiply([1, -2, 5]) == -10 # 1 * -2 * 5 = -10
        multiply([]) == 1
    """
  product = 1
  for num in numbers:
    product *= num
  return product

def divide(numbers: list[int]) -> float:  # Use float for division
    """Divides numbers in a list sequentially from left to right.

    This function performs division on a list of integers, applying the division
    operation from left to right.  For example, given the list [10, 2, 5], the
    function will calculate 10 / 2 / 5.

    Args:
        numbers: A list of integers to be divided.

    Returns:
        The result of the sequential division as a float.

    Raises:
        ZeroDivisionError: If any number in the list *after* the first element
                           is zero, a ZeroDivisionError is raised.  Division by
                           zero is not permitted.

    Returns:
        float: The result of the division. Returns 0.0 if the input list is empty.

    Examples:
        divide([10, 2, 5]) == 1.0  # (10 / 2) / 5 = 5 / 5 = 1.0
        divide([10, 2]) == 5.0      # 10 / 2 = 5.0
        divide([10, 0, 5])  # Raises ZeroDivisionError
        divide([]) == 0.0
    """
    if not numbers:
        return 0.0 # Handle empty list
    if 0 in numbers[1:]: # Check for division by zero
        raise ZeroDivisionError("Cannot divide by zero.")
    result = numbers[0]
    for num in numbers[1:]:
        result /= num
    return result


In [29]:
add([2, 2, 3])

7

In [30]:
multiply([2, 2, 3])

12

In [31]:
simple_math_agent = Agent(
    model=MODEL,
    name="simple_math_agent",
    description="This agent performs basic arithmetic operations (addition, subtraction, multiplication, and division) on user-provided numbers, including ranges.",
    instruction="""
      I can perform addition, subtraction, multiplication, and division operations on numbers you provide.
      Tell me the numbers you want to operate on.
      For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'.
      You can also provide a range: 'Multiply the numbers between 1 and 10'.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[add, subtract, multiply, divide],
)

In [32]:
import random

def caller_factory(root_agent, app_name='App12345', user_id='User12345', session_id=None):
  """create a pre-configured agent caller.

    Args:
        root_agent: The ADK agent to handle conversations
        app_name: Application name (default: 'App12345')
        user_id: User identifier (default: 'User12345')
        session_id: Optional session ID. If None, generates a random one.

    Returns:
        A function that takes a query string and returns agent response events.
  """
  session_service = InMemorySessionService() #  to manage conversation sessions
  if session_id is None:
    suffix = random.randint(100000, 999999)
    session_id = f'{app_name}-{user_id}-{suffix}'
  session = session_service.create_session(
      app_name=app_name, user_id=user_id, session_id=session_id)
  runner = Runner(agent=root_agent, app_name=app_name, session_service=session_service) #Create a Runner instance that will execute the agent
  def _call(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
    return events
  return _call

In [33]:
call = caller_factory(root_agent=simple_math_agent)

In [34]:
pprint_events(call('hello'))

>>> Final Response (1179.707 ms):
Hello! How can I help you with your math today?

------------------------------
Total elapsed time: 1180.529 ms


In [35]:
pprint_events(call('what is three plus 9?'))

+++ Function Calls:
Function Name: add, Args: {'numbers': [3, 9]}
--- Function Responses:
Function Name: add
Function Results: [12]
>>> Final Response (2246.165 ms):
The answer is 12.

------------------------------
Total elapsed time: 2247.162 ms


In [36]:
pprint_events(call('multiply that by 2'))

+++ Function Calls:
Function Name: multiply, Args: {'numbers': [12, 2]}
--- Function Responses:
Function Name: multiply
Function Results: [24]
>>> Final Response (2283.54 ms):
The answer is 24.

------------------------------
Total elapsed time: 2284.392 ms


In [37]:
call = caller_factory(root_agent=simple_math_agent)
pprint_events(call('Here is my math problem that is about apple counting. Let us start saying that I have three apple'))

>>> Final Response (1186.735 ms):
I am designed to perform simple math operations. I can only work with numbers. I cannot count apples.

------------------------------
Total elapsed time: 1187.458 ms


In [38]:
pprint_events(call('Alice gave anoter 2 apples'))

>>> Final Response (1186.433 ms):
I am designed to perform simple math operations. I can only work with numbers.

------------------------------
Total elapsed time: 1187.587 ms


In [39]:
pprint_events(call('how many apples do I have?'))

>>> Final Response (1173.151 ms):
I am designed to perform simple math operations. I can only work with numbers.

------------------------------
Total elapsed time: 1174.095 ms


In [40]:
pprint_events(call('Bob gave me 3 apples. how many apples do I have?'))

>>> Final Response (1165.081 ms):
I am designed to perform simple math operations. I can only work with numbers.

------------------------------
Total elapsed time: 1165.95 ms


In [41]:
pprint_events(call('Alice and Bob do this to me for three more times. how many apples do I have?'))

>>> Final Response (1190.958 ms):
I am designed to perform simple math operations. I can only work with numbers.

------------------------------
Total elapsed time: 1192.15 ms


In [42]:
pprint_events(call('yes your logic is correct'))

>>> Final Response (1162.331 ms):
Okay. I am ready for your math problem.

------------------------------
Total elapsed time: 1163.122 ms


# Agent As Tool

**AgentTool:** You can embed the power of one agent within another by using AgentTool. This allows you to treat an entire agent as a tool within a parent agent. AgentTools are executed in an isolated environment, promoting safety and modularity. Combined with input/output schemas, this enables the creation of sophisticated interactions


Function Tool vs Agent Tool:
 If the execution flow goes from Agent to Tool and it always come back to theinitiation agent vs if the execution flow goes from Agent to **another** agent, the latter agent can decide where the flow goes next.

Concepts:
- AgentTool -- If you want another agent to handle a task and always come back to the caller, make this `Agent` a `Tool` by `AgentTool`

Use case:
- Advanced math agent

In [43]:
agent_math_advanced_instruction = '''
I am an advanced math agent. I handle user query in the below steps:

1. I shall analyse the chat log to understand current question and make a math formula for it.
2. Break down a complex compuation based on arithmetic priority and hand over to simple_math_agent for the calculation.
3. Note that simple_math_agent can only understand numbers, so I need to convert natural language expression of numbers into digits.

<example>
<input> alice gives us 3 apples, bob gives us 5 apples. They do this seven times. Then we eat four apples. How many apples do we have now? </input>
<think> what is (3+5) * 7 -4 </think>
<think>I need to first calculate (3+5) as the highest priority operation.</think>
<call_tool> pass (3+5) to simple_math_agent </call_tool>
<tool_response>8</tool_response>
<think> The question now becomes 8 * 7 - 4, and next highest operation is 8 * 7</think>
<call_tool> pass 8 * 7 to simple_math_agent </call_tool>
<tool_response>56</tool_response>
<think> The question now becomes 56 - 4, and next highest operation is 56 - 4</think>
<call_tool> pass 56 - 4 to simple_math_agent </call_tool>
<tool_response>52</tool_response>
<think>There is a single number, so it is the final answer.</think>
<output>The result of "(3+5) * 7 - 4" is 52</output>
</example>
'''

agent_math_advanced = Agent(
    model=MODEL,
    name="agent_math_advanced",
    description="The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.",
    instruction=agent_math_advanced_instruction,
    tools=[AgentTool(agent=simple_math_agent)],
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [44]:
call = caller_factory(root_agent=agent_math_advanced)

In [45]:
pprint_events(call('who are you?'))

>>> Final Response (1516.341 ms):
I am an advanced math agent. I handle user query in the below steps:

1. I shall analyse the chat log to understand current question and make a math formula for it.
2. Break down a complex compuation based on arithmetic priority and hand over to simple_math_agent for the calculation.
3. Note that simple_math_agent can only understand numbers, so I need to convert natural language expression of numbers into digits.

------------------------------
Total elapsed time: 1517.148 ms


In [46]:
pprint_events(call('what is 1 + (three+2) times 7 '))

+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '3+2'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["The answer is 5.\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '5*7'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["35\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '1+35'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["36\n"]
>>> Final Response (11609.66 ms):
The result of "1 + (three+2) times 7" is 36

------------------------------
Total elapsed time: 11610.519 ms


In [47]:
call = caller_factory(root_agent=agent_math_advanced)

In [48]:
pprint_events(call('add from 1 to 2. then multiply that by 2.'))

+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '1+2'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["3\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '3*2'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["The answer is 6.\n"]
>>> Final Response (7947.823 ms):
The result of "add from 1 to 2. then multiply that by 2" is 6.
------------------------------
Total elapsed time: 7948.531 ms


In [49]:
call = caller_factory(root_agent=agent_math_advanced)
pprint_events(call('how much is 10 / 5 + three times 5?'))

+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '10 / 5'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["The answer is 2.0\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '3 * 5'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["15\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '2.0 + 15'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["I am designed to work with integers. Could you provide me with integers only?\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '2 + 15'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["The answer is 17."]
>>> Final Response (13921.789 ms):
The result of "10 / 5 + three times 5" is 17

------------------------------
Total elapsed time: 13922.685 ms


**NOTE**: This part is very nice, it shows how two agents interact to clarify the input type should be integers and input again.

```
+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2.0 + 15'}
--- inside the function call response...
Function Name: simple_math_agent
Function Results ["I can only work with integers. Could you provide the numbers as integers?\n"]
WARNING:google_genai.types:Warning: there are non-text parts in the response: ['function_call'],returning concatenated text from text parts,check out the non text parts for full response from model.
+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2 + 15'}

```

# Input/Output Format Control

**Input Schema / Output Schema:** Ensure data consistency and validity by defining Pydantic schemas for your agent's input and output. The agent can then verify the input against the schema and guarantee that the generated output conforms to the specified structure using constrained decoding

In [50]:
from typing import List
from pydantic import BaseModel, Field
from google.genai import types

json_response_config = types.GenerateContentConfig(
  response_mime_type="application/json",
)

class OutputSchema(BaseModel):
    original_query: str = Field(description="The original text from user.")
    corrected_text: str = Field(description="The corrected text.")
    errors: List[str] = Field(description="An array of descriptions of each error.")
    explanations: List[str] = Field(description="An array of explanations for each correction.")

json_schema = OutputSchema.model_json_schema()
json_schema

{'properties': {'original_query': {'description': 'The original text from user.',
   'title': 'Original Query',
   'type': 'string'},
  'corrected_text': {'description': 'The corrected text.',
   'title': 'Corrected Text',
   'type': 'string'},
  'errors': {'description': 'An array of descriptions of each error.',
   'items': {'type': 'string'},
   'title': 'Errors',
   'type': 'array'},
  'explanations': {'description': 'An array of explanations for each correction.',
   'items': {'type': 'string'},
   'title': 'Explanations',
   'type': 'array'}},
 'required': ['original_query', 'corrected_text', 'errors', 'explanations'],
 'title': 'OutputSchema',
 'type': 'object'}

In [54]:
# 1. The {json_schema} in instruction is the key for model to follow the schema.
# 2. The output_schema=OutputSchema provides a validation step after model output.

agent_grammar = Agent(
    model=MODEL,
    name='agent_grammar',
    description="This agent corrects grammar mistakes in text provided by children, explains the errors in simple terms, and returns both the corrected text and the explanations.",
    instruction=f"""
        You are a friendly grammar helper for kids.  Analyze the following text,
        correct any grammar mistakes, and explain the errors in a way that a
        child can easily understand.  Don't just list the errors; explain them
        in a paragraph using simple but concise language.

        Output in a JSON object with the below schema:
        {json_schema}
    """,
    output_schema=OutputSchema,
    generate_content_config=json_response_config,
    disallow_transfer_to_parent = True,
    disallow_transfer_to_peers=True
    
    # allow_transfer=False,
)

Conversation Flow:

The session contains multiple interactions where the user asks questions with intentional misspellings, and the agent corrects them. Here are the main interactions:

Response Format: Each agent response is in JSON format containing:

- original_query: The user's input
- corrected_text: The corrected version
- errors: List of error types found
- explanations: Detailed explanations of the corrections
- Technical Details: Each interaction has a unique invocation_id
- Events are timestamped
- The session maintains state and tracks all events


To summarize, this agent:

1. Accepts user input
2. Identifies grammatical and spelling errors
3. Provides corrected versions
4. Explains the corrections
5. Maintains a conversation history
6. The agent is particularly focused on:
7. Spelling corrections (e.g., "ho" → "how", "whut" → "what")
8. Sentence structure (e.g., adding missing "be")
9. Providing educational explanations for each correction

In [55]:
call = caller_factory(root_agent=agent_grammar, app_name='grammar', user_id='User12345', session_id='333')

In [58]:
# session_service.sessions['grammar']['User12345']['333']

In [59]:
pprint_events(call('ho much is three times 5?'))

>>> Final Response (1764.453 ms):
{
  "original_query": "ho much is three times 5?",
  "corrected_text": "How much is three times 5?",
  "errors": [
    "Misspelling",
    "Incomplete sentence"
  ],
  "explanations": [
    "The word 'how' was misspelled. Always double check your spelling to make sure people understand what you are saying!",
    "The query is a question, so it needs a question mark at the end!"
  ]
}
------------------------------
Total elapsed time: 1765.663 ms


In [60]:
pprint_events(call('whut will  2 plus 3?'))

>>> Final Response (1723.596 ms):
{
  "original_query": "whut will  2 plus 3?",
  "corrected_text": "What will 2 plus 3 be?",
  "errors": [
    "Misspelling",
    "Incomplete sentence"
  ],
  "explanations": [
    "The word 'what' was misspelled. It's important to spell words correctly so everyone knows what you mean!",
    "The sentence was not complete. I added 'be' at the end to make it a complete question!"
  ]
}
------------------------------
Total elapsed time: 1724.428 ms


In [62]:
# session_service.sessions['grammar']['User12345']['333']

In [84]:
!gcloud storage cp -r gs://adk_training/sample_agents/ .

Copying gs://adk_training/sample_agents/agent_dice/__init__.py to file://./sample_agents/agent_dice/__init__.py
Copying gs://adk_training/sample_agents/agent_dice/__pycache__/__init__.cpython-311.pyc to file://./sample_agents/agent_dice/__pycache__/__init__.cpython-311.pyc
Copying gs://adk_training/sample_agents/agent_dice/__pycache__/agent.cpython-311.pyc to file://./sample_agents/agent_dice/__pycache__/agent.cpython-311.pyc
Copying gs://adk_training/sample_agents/agent_dice/agent.py to file://./sample_agents/agent_dice/agent.py
Copying gs://adk_training/sample_agents/agent_teaching_assistant/__init__.py to file://./sample_agents/agent_teaching_assistant/__init__.py
Copying gs://adk_training/sample_agents/agent_teaching_assistant/__pycache__/__init__.cpython-311.pyc to file://./sample_agents/agent_teaching_assistant/__pycache__/__init__.cpython-311.pyc
Copying gs://adk_training/sample_agents/agent_teaching_assistant/__pycache__/agent.cpython-311.pyc to file://./sample_agents/agent_tea

In [79]:
!ls

ADK_Training_30min.ipynb  Toolbox_in_Agent_ADK.ipynb


In [80]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇npm notice
npm notice New major version of npm available! 10.9.0 -> 11.3.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.3.0
npm notice To update run: npm install -g npm@11.3.0
npm notice
⠇

In [ ]:
# Run this command from the shell (lower left corner of CoLab)
# npx localtunnel --port 8501


In [81]:
# Run this in the CoLab to get the password, which is basically an IP address.
!curl https://loca.lt/mytunnelpassword

34.135.111.231

In [ ]:
# Run this to launch the UI of ADK later
!cd sample_agents && adk web

INFO:     Started server process [2434745]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http://localhost:8000.                         |
+-----------------------------------------------------------------------------+

INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     132.147.101.181:5346 - "GET / HTTP/1.1" 307 Temporary Redirect
INFO:     132.147.101.181:5346 - "GET /main-HWIBUY2R.js HTTP/1.1" 200 OK
INFO:     132.147.101.181:5346 - "GET /list-apps?relative_path=./ HTTP/1.1" 200 OK
2025-05-04 15:19:23,388 - INFO - fast_api.py:313 - New session created
INFO:     132.147.101.181:5346 - "POST /apps/agent_dice/users/user/sessions HTTP/1.1" 200 OK
INFO:  